In [1]:
import numpy as np
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from astropy import units as u
import bisect as bs

# 40054 zLDPs, lets see how many slits there are


In [2]:
macrofnamelist = 'slit_masks_dj/slit_file_list.dat'
with open(macrofnamelist, 'r') as fin:
    fnames = fin.readlines()
fnames = [f_.split()[0] for f_ in fnames]

In [3]:
slit_mask = Table.read('slit_masks_dj/' + fnames[0], format='ascii')
for fname in fnames[1:]:
    temp_table = Table.read('slit_masks_dj/' + fname, format='ascii')
    slit_mask = vstack([slit_mask, temp_table])
print(len(slit_mask))

67185


# Load the photometry catalog

In [4]:
photcat = Table().read('personal_catalogs/megacat.fits')

photcat['zLDP'] = -1.
photcat['zLDPerr'] = -1.
photcat['Q'] = -1
photcat['zLDP_good'] = -1
photcat['slit_distance'] = -1.
photcat['zSpec'] = -1.
photcat['zSpec_Q'] = -1.
photcat['spec_type'] = ''
photcat['found_slit'] = 0

photcat['zLDP'].mask = True
photcat['zLDPerr'].mask = True
photcat['Q'].mask = True
photcat['zLDP_good'].mask = True
photcat['slit_distance'].mask = True
photcat['zSpec'].mask = True
photcat['zSpec_Q'].mask = True
photcat['spec_type'].mask = True

photcat.sort('ra')
photcat['uid'] = np.arange(0, len(photcat['ra']), 1)

In [5]:
print(photcat.colnames)

['field', 'ids', 'ra', 'dec', 'x', 'y', 'ebv', 'fwhmR', 'fB1', 'fB1err', 'fB2', 'fB2err', 'fB3', 'fB3err', 'fBiso', 'fBisoerr', 'fBauto', 'fBautoerr', 'fV1', 'fV1err', 'fV2', 'fV2err', 'fV3', 'fV3err', 'fViso', 'fVisoerr', 'fVauto', 'fVautoerr', 'fR1', 'fR1err', 'fR2', 'fR2err', 'fR3', 'fR3err', 'fRiso', 'fRisoerr', 'fRauto', 'fRautoerr', 'fI1', 'fI1err', 'fI2', 'fI2err', 'fI3', 'fI3err', 'fIiso', 'fIisoerr', 'fIauto', 'fIautoerr', 'fz1', 'fz1err', 'fz2', 'fz2err', 'fz3', 'fz3err', 'fziso', 'fzisoerr', 'fzauto', 'fzautoerr', 'fK1', 'fK1err', 'fK2', 'fK2err', 'fK3', 'fK3err', 'fKiso', 'fKisoerr', 'fKauto', 'fKautoerr', 'zLDP5_7', 'Q5_7', 'zphot', 'zphot_errUP', 'zphot_errLO', 'class_StarR', 'iso_area', 'major_ax', 'minor_ax', 'theta', 'sexflagB', 'sexflagV', 'sexflagR', 'sexflagI', 'sexflagz', 'sexflagK', 'wK', 'zLDP', 'zLDPerr', 'Q', 'zLDP_good', 'slit_distance', 'zSpec', 'zSpec_Q', 'spec_type', 'found_slit', 'uid']


# For each slit, find the photometry entry.

In [6]:
slit_coords = SkyCoord(ra=slit_mask['ra']*u.degree, dec=slit_mask['dec']*u.degree)
phot_coords = SkyCoord(ra=photcat['ra']*u.degree, dec=photcat['dec']*u.degree)

In [7]:
# the nth element of idx is in the index in phot_coords that best matches the nth element in slit_coords
idx, d2d, d3d = slit_coords.match_to_catalog_sky(phot_coords)

In [8]:
print('Slits={}'.format(len(d2d)))
print('Matched under 1"={}'.format(len(d2d[np.where(d2d < 1.*u.arcsecond)])))
print('Unique matches={}'.format(len(set(idx[np.where(d2d < 1.*u.arcsecond)]))))

Slits=67185
Matched under 1"=54836
Unique matches=46953


In [9]:
idx_lt1as = idx[np.where(d2d < 1.*u.arcsecond)]
photcat['found_slit'][idx_lt1as] = 1

In [10]:
print("Num of obj on slits = {}".format(np.sum(photcat['found_slit'])))

Num of obj on slits = 46953


# Load the LDP data and compare to photometry

In [11]:
primus_table = Table().read('personal_catalogs/primus_fors2.csv', format='csv')

In [12]:
print(primus_table.colnames)
print(len(primus_table))

['ra', 'dec', 'zLDP', 'zLDPerr', 'Q', 'zLDP_good', 'ra_slit', 'dec_slit', 'slit_distance', 'uid', 'zSpec', 'zSpec_quality', 'spec_type', 'zspec_dist_asec']
54131


In [13]:
ldp_coords = SkyCoord(ra=primus_table['ra_slit']*u.degree, dec=primus_table['dec_slit']*u.degree)

In [14]:
# Non-vectorized method
# The nth element of ldpidx is the index of phot_coords that best matches the nth element of ldp_coords
#ldpidx, ldpd2d, idxd3d = ldp_coords.match_to_catalog_sky(phot_coords)
ldpidx, ldpd2d, ldpd3d = phot_coords.match_to_catalog_sky(ldp_coords)
# Only keep matches when slit-obj sep is less than 1 arcsecond
#close_ldpidx = ldpidx[np.where(ldpd2d < 1.*u.arcsecond)[0]]
#close_primus_table = primus_table[np.where(ldpd2d < 1.*u.arcsecond)[0]]

In [15]:
photcat['slit_distance'] = ldpd2d.arcsecond
photcat['zLDP'] = primus_table['zLDP'][ldpidx]
photcat['zLDPerr'] = primus_table['zLDPerr'][ldpidx]
photcat['Q'] = primus_table['Q'][ldpidx]
photcat['zLDP_good'] = primus_table['zLDP_good'][ldpidx]
photcat['zSpec'] = primus_table['zSpec'][ldpidx]
photcat['zSpec_Q'] = primus_table['zSpec_quality'][ldpidx]
photcat['spec_type'] = primus_table['spec_type'][ldpidx]

In [16]:
# Non-vectorized method
#def match_row(tphotcat, row, temp_id):
#    tphotcat[temp_id]['zLDP'] = row['zLDP']
#    tphotcat[temp_id]['zLDPerr'] = row['zLDPerr']
#    tphotcat[temp_id]['Q'] = row['Q']
#    tphotcat[temp_id]['zLDP_good'] = row['zLDP_good']
#    tphotcat[temp_id]['slit_distance'] = row['slit_distance']
#    tphotcat[temp_id]['zSpec'] = row['zSpec']
#    tphotcat[temp_id]['zSpec_Q'] = row['zSpec_quality']
#    return

In [17]:
# Non-vectorized method
#for ii in range(len(close_ldpidx)):
#    match_row(photcat, close_primus_table[ii], close_ldpidx[ii])

# Photcat now has LDP/Specz's within it and a flag if there was a slit placed upon it

In [18]:
# Make a subtable only containing objects with slits placed upon them.
subphot = photcat[np.where(photcat['found_slit'] == 1)[0]]
print(len(subphot))

46953


# Calculate mags from fluxes

In [19]:
def flux2mag(f):
    return -2.5*np.log10(f) + 23.9

def calc_mag(photo, filt):
    flux = 'f'+filt
    mag = flux2mag(photo[flux])
    #mag = [flux2mag(f) for f in photo[flux]]
    photo[filt] = mag
    #photo[filt][np.isnan(mag)] = 101.
    photo[filt].mask = np.isnan(mag)

    return

In [20]:
filterlist = ['B1', 'V1', 'R1', 'I1', 'z1',
              'B2', 'V2', 'R2', 'I2', 'z2',
              'B3', 'V3', 'R3', 'I3', 'z3',
              'Bauto', 'Vauto', 'Rauto', 'Iauto', 'zauto']

for filt in filterlist:
    calc_mag(subphot, filt)

/Users/matthewkirby/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  


In [21]:
final_table = subphot['field', 'ids', 'ra', 'dec', 'x', 'y',
                          'B1', 'V1', 'R1', 'I1', 'z1',
                          'B2', 'V2', 'R2', 'I2', 'z2',
                          'B3', 'V3', 'R3', 'I3', 'z3',
                          'Bauto', 'Vauto', 'Rauto', 'Iauto', 'zauto',
                          'zLDP5_7', 'Q5_7', 'zphot', 'zphot_errUP', 'zphot_errLO', 'class_StarR',
                          'zLDP', 'zLDPerr', 'Q', 'zLDP_good', 'slit_distance', 'zSpec', 'zSpec_Q', 'spec_type']

In [22]:
final_table.write('personal_catalogs/slits_phot_zs_cutonslitdist.csv', format='csv', overwrite=True)

In [23]:
print(len(subphot))
print(len(subphot[np.where((subphot['slit_distance'] < 1.0)&(subphot['Q']==4)&(subphot['Rauto']<22.9))]))

46953
18021
